In [1]:
import psycopg2
import re

In [2]:
def connect():
    def executeScriptsFromFile(filename):
        # Open and read the file as a single buffer
        fd = open(filename, 'r')
        sqlFile = fd.read()
        fd.close()

        # all SQL commands (split on ';')
        sqlCommands = sqlFile.split(';')
        # Remove last element which is empty
        sqlCommands.pop(-1) 
        
        # Execute every command from the input file
        for command in sqlCommands:
            # This will skip and report errors
            # For example, if the tables do not yet exist, this will skip over
            # the DROP TABLE commands
            try:
                print("command: ", command)
                cur.execute(command)
            except (Exception, psycopg2.DatabaseError) as error:
                print(error)
                
    def transformCheckConstraintsFromFile(filename1, filename2):
        # get all check constraints
        fd = open(filename1, 'r')
        sqlFile = fd.read()
        fd.close()
        str_list = re.findall(r"(?<=CHECK).*", sqlFile)
        
        # TODO: is there a less hard-coded way to find the table name?
        table_name = re.findall(r"(?<=CREATE TABLE ).*", sqlFile)
        table_name = table_name[0].split()[0]
        print("table_name: ", table_name)

        constraints_list = []
        for i in range(len(str_list)):
            constraint = str_list[i]
            constraint = constraint[constraint.index('(')+1:constraint.index(')')].split()
            constraints_list.append(constraint)
        print("constraints_list: ", constraints_list)

        # transform each check constraint to a trigger
        fd = open(filename2, 'r')
        skeleton = fd.read()
        fd.close()
        triggers_list = []
        for constraint in constraints_list:
            trigger = skeleton.format(constraint[0], 
                                table_name,
                                constraint[0], 
                                constraint[0], 
                                constraint[0], 
                                constraint[1], 
                                constraint[2], 
                                constraint[0],
                                constraint[0],
                                table_name,
                                constraint[0])
            print("trigger: ", trigger)
            print("==================================================\n")
            triggers_list.append(trigger)
        
        # TODO: copy create_table.sql to a new sql file,
        # and remove all CHECK sections
        
        # TODO: insert all triggers to the new file
        
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
            host="localhost",
            database="cs5421",
            user="postgres",
            password="common2021")# Change the database name & credentials accordingly!

        # create a cursor
        cur = conn.cursor()
        
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
        # execute commands from the .sql file
#         executeScriptsFromFile("create_table.sql")
#         executeScriptsFromFile("query1.sql")
        
        # for create table...
        # transform the check constraints from the .sql file
        transformCheckConstraintsFromFile("create_table.sql", "function_trigger_skeleton.txt")
        
        # TODO: for alter table...
        
        # TODO: compare performance of check vs.trigger
        
        # close the communication with the PostgreSQL
        cur.close()
        
        # commit the changes
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()
    


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.2 (Ubuntu 15.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',)
table_name:  employees
constraints_list:  [['birth_date', '>', "'1900-01-01'"], ['joined_date', '>', 'birth_date'], ['salary', '>', '0']]
trigger:  DROP TRIGGER IF EXISTS verify_birth_date on employees;
DROP FUNCTION IF EXISTS birth_date;

CREATE FUNCTION birth_date()
RETURNS trigger AS $BODY$
BEGIN
IF new.birth_date > '1900-01-01' THEN
    RETURN NEW;
ELSE 
    RAISE EXCEPTION 'Invalid birth_date';
END IF;
END;
$BODY$
LANGUAGE 'plpgsql';

CREATE TRIGGER verify_birth_date BEFORE INSERT OR UPDATE ON employees
            FOR EACH ROW EXECUTE PROCEDURE birth_date();

trigger:  DROP TRIGGER IF EXISTS verify_joined_date on employees;
DROP FUNCTION IF EXISTS joined_date;

CREATE FUNCTION joined_date()
RETURNS trigger AS $BODY$
BEGIN
IF new.joined_date > birth_date THEN
    RETURN NEW;
ELSE